In [17]:
import sys
sys.path.append('C:/Users/Kai/Desktop/vis_scBERT/DNABERT_2/')
from get_gene_name_and_seq import *
import pickle

# Get NT sequence from serval patheways of cmiu

In [4]:
raw='''
00190  Oxidative phosphorylation
00680  Methane metabolism
00910  Nitrogen metabolism
00920  Sulfur metabolism
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

In [8]:
for k,v in cmiu_dic.items():
    print(k, len(v))

cmiu00190 12
cmiu00680 28
cmiu00910 7
cmiu00920 6


# Build data for "gene sense" task

In [8]:
raw='''
cmiu00010  	Glycolysis / Gluconeogenesis
cmiu00051  	Fructose and mannose metabolism
cmiu00052  	Galactose metabolism
cmiu00500  	Starch and sucrose metabolism
cmiu00520  	Amino sugar and nucleotide sugar metabolism
cmiu00521  	Streptomycin biosynthesis
cmiu01100  	Metabolic pathways
cmiu01110  	Biosynthesis of secondary metabolites
cmiu01120  	Microbial metabolism in diverse environments
cmiu01200  	Carbon metabolism
cmiu01250  	Biosynthesis of nucleotide sugars
'''
pathway_list = clean_raw_pathway_str(raw, 'cmiu')
cmiu_dic = pathway2NTseq_cmiu(pathway_list)

Getting NT sequences: 100%|██████████| 11/11 [03:19<00:00, 18.17s/it]


In [ ]:
cmiu_dic

## TODO: build sentences in neighbors of a gene
- shuffle one single leaf node
- build a larger pseudo dataset

In [4]:
with open("B1H56_02580.pkl", 'rb') as f:
    cmiu_dic = pickle.load(f)

cmiu_dic

{'cmiu00010': {'cmiu:B1H56_02580': 'ATGAAAGGAACGCATATGCTGACTGCAAACGAGTTTATCAGGAAACATAAAATGGAATACACGTCCATTGACGTAAAAAACGAGATGGACACTTACATCGATGAAATGAAAAAGGGATTGTGCGGCGAGAAGAGTTCGCTGCTGATGATCCCTTCCTATATTACGCTGAAAAGCGAAGTGAAGCGGGAAAAACCGGTGATCTGCGTAGACGCGGGCGGGACGAATCTGCGCGTTGCGGTGGCGAAATTTTCCGAGGACGGCTCTTTTTGCACGGAGGAATTCCAGCGGTATTTGATGCCGGGTGTGGAAAAGGAACTTGAGGCCGAAGAATTTTTTGATATTTTGGCGGATTATATCCTGCCTTTTACGAAAATCACGAAAAATATTGTGATTTCATTCGCTTACCGGGCAAAAATACTGCCCGATATCGACTGCGAGATCGTAGAAATCACCAAAGAAGTCAAAGTGAAAAATGCAGGCGGAAAACTGCTGGGCAGAGAGATTTGCGCGGCCTTGGCGAAGAAGGGCGCGGACGGCTGTAATATCGTTGTGGTAAACGACTCCGTGGCAACGGCGCTTGCCGGAAAAGCGGAAAAACTGAACGACGGATACGGCTCCTTTACGGGGACGATACTTGGTACGGGAAGCAACAGCTGCTATATTGAATATATGCGCAACATCGCGAAGCTTGAAGGCGGCGAAGAGGGCCTGATGGTGATTAATACGGAGGCGGGCAGCTATAATAAGGTGCCGCGCAGCGATATCGATATTGCCTATGACGAAAGCACACAAAACCCTGGGATCGGGGTGTTTGAGAAAATGACCAGCGGCGGCTACTTGGGCCCCCTGTGTGATTTTACCCTGCGCACGGCGGCGCGGGAAGGTGTCTTTGAAAAAGGATTTTCCATGGCAGGGACGGTCTCGACCGTCGACGTGAACGCTTTCCTGACGGACGGGTCGGGCG

In [27]:
import requests
from bs4 import BeautifulSoup

def get_tidy_brite(brite_name):
    '''
    *Input*: 
    brite_name: str, e.g. cmiu04131 
    
    *Output*: 
    soup1: str, soup2: dict
    '''
    url = "https://rest.kegg.jp/get/br:" + brite_name
    r = requests.get(url)
    soup1 = BeautifulSoup(r.text, 'html.parser')
    soup1 = soup1.text
    r = requests.get(url+'/json')
    soup2 = r.json()
    return soup1 ,soup2

In [199]:
def find_neighbors(gene_name:str, brite_str:str):
    '''
    **Inputs**:
    - gene_name: str, e.g. B1H56_02580
    - brite_str: str, e.g. brite_str, _ = get_tidy_brite('cmiu00001')
    
    **Outputs**:
    - neighbors: dict, keys are the hierarchies, values are the gene names. 
    hier be like '09100 Metabolism -> 09101 Carbohydrate metabolism -> 00500 Starch and sucrose metabolism [PATH:cmiu00500] -> B1H56_01825 carbohydrate kinase\tK00847 E2.7.1.4; fructokinase [EC:2.7.1.4]'
    '''
    neighbors = {}
    level = ['A','B']
    hier = ''
    names = []
    
    for i in brite_str.split('\n'):
        if i:
            tmp = i[1:].strip().split()
            
            flag = False # flag to indicate whether it is time to update the hier
            if i[0] in ('A', 'B', 'C', 'D', 'E', 'F', 'G'):
                level[1] = i[0]
                flag = True
            if level[1] == 'A':
                hier = i[1:].strip()
            elif flag:
                if level[1]>level[0]:
                    hier += ' -> ' + i[1:].strip()
                    if tmp and tmp[0].startswith(gene_name[:3]):
                        names.append(tmp[0])
                if level[1]==level[0]:
                    # back to the previous level
                    hier = ''.join([i+' -> ' for i in hier.split(' -> ')[:-1]]) + i[1:].strip()
                    if tmp and tmp[0].startswith(gene_name[:3]):
                        names.append(tmp[0])
                if level[1]<level[0]:
                    # back to the previous level twice
                    if gene_name in names:
                        neighbors[hier] = names
                    hier = ''.join([i+' -> ' for i in hier.split(' -> ')[:-2]]) + i[1:].strip()
                    names = []
            level[0] = level[1]
    
    return neighbors

In [200]:
brite_str, _ = get_tidy_brite('cmiu00001')
a = find_neighbors('B1H56_02580', brite_str)

In [201]:
a

{'09100 Metabolism -> 09101 Carbohydrate metabolism -> 00010 Glycolysis / Gluconeogenesis [PATH:cmiu00010] -> B1H56_12425 pckA; phosphoenolpyruvate carboxykinase (ATP)\tK01610 pckA; phosphoenolpyruvate carboxykinase (ATP) [EC:4.1.1.49]': ['B1H56_02580',
  'B1H56_05240',
  'B1H56_03055',
  'B1H56_07930',
  'B1H56_02545',
  'B1H56_12605',
  'B1H56_00960',
  'B1H56_11715',
  'B1H56_06330',
  'B1H56_08575',
  'B1H56_05180',
  'B1H56_08040',
  'B1H56_14070',
  'B1H56_00975',
  'B1H56_08035',
  'B1H56_02480',
  'B1H56_05080',
  'B1H56_03225',
  'B1H56_02845',
  'B1H56_07355',
  'B1H56_13745',
  'B1H56_07350',
  'B1H56_00500',
  'B1H56_09305',
  'B1H56_01335',
  'B1H56_01340',
  'B1H56_01465',
  'B1H56_02180',
  'B1H56_02515',
  'B1H56_13645',
  'B1H56_04770',
  'B1H56_09480',
  'B1H56_02750',
  'B1H56_12425'],
 '09100 Metabolism -> 09101 Carbohydrate metabolism -> 00051 Fructose and mannose metabolism [PATH:cmiu00051] -> B1H56_00190 rpiB; ribose 5-phosphate isomerase B\tK01808 rpiB; ribose 5

In [242]:
gene_name = 'B1H56_02580'

def get_nt_seq_tidy(gene_name, prefix='cmiu:'):
    url = "https://rest.kegg.jp/get/" + prefix + gene_name
    r = requests.get(url)
    s = BeautifulSoup(r.text, 'html.parser').text
    split_soup = s.split('\n')
    for i in range(len(split_soup)):
        if split_soup[i].startswith('NT'):
            raw_s = ''.join(split_soup[i+1:])
            return(re.sub(r"[ /]", "", raw_s)).upper()
    
    return None

In [244]:
s = get_nt_seq_tidy('B1H56_02580')
len(s)

1272

### Build sentences of each nodes within one BRITE hier

In [18]:
def build_sentences(gene_name, neighbors, prefix='cmiu:', sentence_length=None):
    
    def _single_sentence_group(gene_name, local_neighbor, prefix):
        sent = []
        indices = []
        n_sentences = _get_n_sentences(local_neighbor, sentence_length)
        for t in range(n_sentences):
            if len(local_neighbor) < 5:
                names = random.sample(local_neighbor, len(local_neighbor))
            else:
                if len(local_neighbor) < 50:
                    vol = int(len(local_neighbor) * .6)
                    while True:
                        names = random.sample(local_neighbor, vol)
                        if gene_name in names:
                            break
            nt_seqs = []
            for i in names:
                nt_seq = get_nt_seq_tidy(i, prefix)
                if nt_seq:
                    nt_seqs.append(nt_seq)
                    indices.append(local_neighbor.index(i))
            tmp_sentence = ' '.join(nt_seqs)
            sent.append(tmp_sentence)
        return sent, indices

    def _get_n_sentences(local_neighbor, sentence_length):
        # TODO: implement a better way to determine the number of sentences
        if len(local_neighbor) < 5:
            return 1
        return 5

    sentences = {}
    for k, v in tqdm(neighbors.items()):
        sent, indices = _single_sentence_group(gene_name, v, prefix)
        sentences[k] = (sent, indices)
    return sentences

In [19]:
# get all neighbors of a gene in a BRITE hierarchy
brite_str, _ = get_tidy_brite('cmiu00001')
neighbors = find_neighbors('B1H56_02580', brite_str)

sentences = build_sentences('B1H56_02580', neighbors, prefix='cmiu:', sentence_length=5)

100%|██████████| 8/8 [07:31<00:00, 56.49s/it] 


In [20]:
with open("B1H56_02580_cmiu00001_sentences.pkl", 'wb') as f:
    pickle.dump(sentences, f)

##### visualizing

In [21]:
sentences

{'09100 Metabolism -> 09101 Carbohydrate metabolism -> 00010 Glycolysis / Gluconeogenesis [PATH:cmiu00010] -> B1H56_12425 pckA; phosphoenolpyruvate carboxykinase (ATP)\tK01610 pckA; phosphoenolpyruvate carboxykinase (ATP) [EC:4.1.1.49]': (['...'],
  [4,
   5,
   19,
   7,
   2,
   8,
   17,
   28,
   0,
   32,
   21,
   26,
   15,
   23,
   31,
   25,
   18,
   22,
   1,
   3,
   14,
   19,
   11,
   23,
   1,
   13,
   32,
   7,
   24,
   20,
   10,
   31,
   25,
   27,
   22,
   6,
   16,
   4,
   0,
   8,
   10,
   17,
   9,
   29,
   27,
   2,
   7,
   28,
   31,
   4,
   0,
   24,
   1,
   33,
   14,
   8,
   12,
   3,
   16,
   23,
   23,
   11,
   13,
   32,
   26,
   20,
   29,
   2,
   15,
   3,
   16,
   21,
   24,
   12,
   28,
   33,
   17,
   30,
   6,
   0,
   9,
   22,
   14,
   15,
   26,
   0,
   2,
   17,
   10,
   13,
   1,
   19,
   24,
   31,
   18,
   28,
   32,
   27,
   6,
   29]),
 '09100 Metabolism -> 09101 Carbohydrate metabolism -> 00051 Fructose and mannose